In [1]:
# pytorch neural network
import gym
import numpy as np
import math
import time
import argparse
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
from google.colab.patches import cv2_imshow
import random
import tqdm


In [2]:
!pip install wandb
!wandb login

wandb: Currently logged in as: arubittu (use `wandb login --relogin` to force relogin)


In [3]:
import wandb
wandb.init(project="DQN-Training", entity="arubittu")

wandb: Currently logged in as: arubittu (use `wandb login --relogin` to force relogin)


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
pip install pyglet==1.5.11


  Using cached pyglet-1.5.11-py3-none-any.whl (1.1 MB)
  Attempting uninstall: pyglet
    Found existing installation: pyglet 1.5.0
    Uninstalling pyglet-1.5.0:
      Successfully uninstalled pyglet-1.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gym 0.17.3 requires pyglet<=1.5.0,>=1.4.0, but you have pyglet 1.5.11 which is incompatible.


In [6]:
# dqn nn
class DQN(nn.Module):

  def __init__(self):
    super(DQN, self).__init__()
    
  
    self.cnn1 = nn.Conv2d(4,32,8,4)
    self.cnn2 = nn.Conv2d(32,64,4,2)
    self.cnn3 = nn.Conv2d(64,64,3,1)
    
    self.fc1 = nn.Linear(3136,512)
    self.out = nn.Linear(512,4)
    self.relu = nn.ReLU()

  def forward(self, x):

    x = self.relu(self.cnn1(x))
    x = self.relu(self.cnn2(x))
    x = self.relu(self.cnn3(x))
    x = x.view(x.size(0),-1)
    x = self.fc1(x)
    x = self.out(x)

    return x 
  
  def linear_in_features(self,x):
    in_features = x.view(1,-1).size(1)
    return in_features


#############
net = DQN()
a = torch.randn((1,4,84,84))
v = net(a)
    

In [7]:
!python -m atari_py.import_roms '/content/drive/MyDrive/rom'

copying adventure.bin from ROMS/Adventure (1980) (Atari, Warren Robinett) (CX2613, CX2613P) (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin from ROMS/Air Raid (Men-A-Vision) (PAL) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/air_raid.bin
copying alien.bin from ROMS/Alien (1982) (20th Century Fox Video Games, Douglas 'Dallas North' Neubauer) (11006) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/alien.bin
copying amidar.bin from ROMS/Amidar (1982) (Parker Brothers, Ed Temple) (PB5310) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/amidar.bin
copying assault.bin from ROMS/Assault (AKA Sky Alien) (1983) (Bomb - Onbase) (CA281).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/assault.bin
copying asterix.bin from ROMS/Asterix (AKA Taz) (07-27-1983) (Atari, Jerome Domurat, Steve Woita) (CX2696) (Prototype).bin to /usr/local/lib/python3.7/dist-packages/a

In [8]:
%%bash

# install required system dependencies
apt-get install -y xvfb x11-utils

# install required python dependencies (might need to install additional gym extras depending)
pip install gym[box2d]==0.17.* pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

Reading package lists...
Building dependency tree...
Reading state information...
x11-utils is already the newest version (7.7+3build1).
xvfb is already the newest version (2:1.19.6-1ubuntu4.9).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
  Using cached pyglet-1.5.0-py2.py3-none-any.whl (1.0 MB)
  Attempting uninstall: pyglet
    Found existing installation: pyglet 1.5.11
    Uninstalling pyglet-1.5.11:
      Successfully uninstalled pyglet-1.5.11


In [9]:
import pyvirtualdisplay


_display = pyvirtualdisplay.Display(visible=False,  # use False with Xvfb
                                    size=(1400, 900))
_ = _display.start()

In [ ]:
# breakout environment and preprocessing code
env = gym.make("Breakout-v0")

print("Observation Space: ", env.observation_space)
print("Action Space       ", env.action_space)


obs = env.reset()

for i in range(1000):
    action = torch.randint(0, 4, (1,))
    obs, reward, done, info = env.step(action)
    #print(reward,action,float(done))
    env.render()
    time.sleep(0.01)
    #plt.imshow(obs)
env.close()

Observation Space:  Box(0, 255, (210, 160, 3), uint8)
Action Space        Discrete(4)


In [ ]:
class replayBuffer():
  def __init__(self,max_elements,batch_size,frame_shape = [4,84,84]):
    self.dataset = []
    self.N = max_elements # max touples
    self.batch_size = batch_size

    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    self.frames = np.zeros([self.N] + frame_shape, dtype=np.uint8)
    self.nextframes = np.zeros([self.N] + frame_shape, dtype=np.uint8)
    self.actions = np.zeros([self.N, 1], dtype=np.uint8)
    self.rewards = np.zeros([self.N, 1], dtype=np.float32)  # we need extra precision for rewards
    self.dones = np.zeros([self.N, 1], dtype=np.uint8)

    self.current_free_index = 0

  def getBatch(self):

    random_unique_indices = random.sample(range(self.current_free_index-1), self.batch_size)
    states = torch.from_numpy(self.frames[random_unique_indices]).to(self.device).float()
    nextStates = torch.from_numpy(self.nextframes[random_unique_indices]).to(self.device).float()
    actions = torch.from_numpy(self.actions[random_unique_indices]).to(self.device).long()
    rewards = torch.from_numpy(self.rewards[random_unique_indices]).to(self.device)
      # Float is needed because we'll be multiplying Q values with done flags (1-done actually)
    dones = torch.from_numpy(self.dones[random_unique_indices]).to(self.device).float()
    return states,nextStates,actions,rewards,dones

  def addData(self,phi_t, action, reward, phi_t2,done):
    if self.current_free_index >= self.N:
      np.roll(self.frames, -1,0)
      np.roll(self.nextframes, -1,0)
      np.roll(self.actions, -1,0)
      np.roll(self.rewards, -1,0)
      np.roll(self.dones, -1,0)
      self.current_free_index = self.current_free_index -1
      self.frames[self.current_free_index] = phi_t.cpu()
      self.nextframes[self.current_free_index] = phi_t2.cpu()
      self.actions[self.current_free_index] = action
      self.rewards[self.current_free_index] = reward
      self.dones[self.current_free_index] = done
      self.current_free_index +=1

    elif self.current_free_index <self.N:
        
      self.frames[self.current_free_index] = phi_t.cpu()
      self.nextframes[self.current_free_index] = phi_t2.cpu()
      self.actions[self.current_free_index] = action
      self.rewards[self.current_free_index] = reward
      self.dones[self.current_free_index] = done
      self.current_free_index +=1
    return
  



In [ ]:
# exp. replay algorithm
class agent():
  def __init__(self,pre_trained_model = False):
    torch.random.manual_seed(0)
    np.random.seed(0)
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # init env
    env = gym.make("Breakout-v0")
    self.original_shape = env.observation_space.shape
    self.num_actions = env.action_space.n

    # init epsilon lr, loss func , optim , target and main NN
    self.net = DQN()
    self.targetNet = DQN()
    self.net.to(self.device)
    self.targetNet.to(self.device)
    
    if pre_trained_model == True:
      self.net_weights = self.load_data()
    else:
      self.net_weights = self.net.state_dict()
    self.targetNet_weights = self.net_weights
    self.targetNet.load_state_dict(self.net_weights)
    self.loss =  nn.SmoothL1Loss()
    self.optimizer = torch.optim.Adam(self.net.parameters(), lr = 0.0001, weight_decay=0)
        
    
    # 4 img tensor
    self.net_input = []

    # train params
    self.train_episodes = 100
    self.test_episodes = 100
    self.warmup_frames = 4
    self.batch_size = 32
    self.epsilon = 0.99
    self.discount = 0.99
    self.C = 1000
    self.stepcount = 0

    #replay buffer
    self.buffer = replayBuffer(20000,self.batch_size)

    #log rewards/loss
    self.loss_log = []
    self.train_rewards = []
    
  def preprocess_state(self,obs):
    # return 1x84x84 img
    obs = obs/255.0
    obs = obs.astype('uint8')
    obs = np.asarray(obs)
    #print(obs.shape,type(obs))
    obs = cv2.cvtColor(obs,cv2.COLOR_BGR2GRAY)
    obs = cv2.resize(obs, (84,84), interpolation = cv2.INTER_AREA)
    obs = torch.tensor(obs).view(1,84,84)
    return obs.to(self.device).float()

  def take_action(self,state):
    if np.random.random() <self.epsilon:
      action = torch.randint(0,4,(1,))
    else:
      with torch.no_grad():
        action = torch.argmax(self.net(torch.unsqueeze(state,0)))
    return action.item()

  def train(self):
    self.reward_list = []
    for episodes in tqdm.tqdm(range(self.train_episodes)):
      self.epsilon = 0.99*(1.1 - 5/self.train_episodes)
      obs = env.reset()
      obs = self.preprocess_state(obs)
      self.net_input = obs
      episode_reward = 0

      while len(self.net_input) != self.warmup_frames:
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        obs = self.preprocess_state(obs)
        self.net_input = torch.cat((self.net_input, obs), dim = 0)   
        
      while done == False:
        #print('ok')
        phi_t = self.net_input
        action = self.take_action(phi_t)
        nxt_obs, reward, done, info = env.step(action)
        episode_reward += reward

        preprocesed_next_obs = self.preprocess_state(nxt_obs)
        self.net_input = self.net_input[1:] # remove earliest frame
        self.net_input = torch.cat((self.net_input, preprocesed_next_obs), dim = 0) # add latest
        phi_t2 = self.net_input

        #transition = [phi_t, action, reward, phi_t2, float(done)]
        self.buffer.addData(phi_t, action, reward, phi_t2, float(done))
        # if more then 32 data points added
        
        if self.stepcount > 32:  

          states,next_states,actions,rewards,dones = self.buffer.getBatch()
          
          self.learn(states,next_states,actions,rewards,dones)

        self.stepcount += 1
        

        if self.stepcount%self.C == 0:
          self.targetNet.load_state_dict(self.net_weights)
          self.save_data()
        
          
      self.reward_list.append(episode_reward)
    self.log()

    return

  def learn(self,states,next_states,actions,rewards,dones):
    with torch.no_grad():
      target_maxQ = self.targetNet(next_states).max(dim=1, keepdim=True)[0]
      targets = rewards + self.discount*(1 - dones)*target_maxQ

    current_state_q_values = self.net(states).gather(dim=1, index=actions) 
    loss = self.loss(current_state_q_values, targets)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.net_weights = self.net.state_dict()
    wandb.log({"loss": loss})
    wandb.watch(self.net)
    
    return loss
  
  def test(self):
    self.net.load_state_dict(self.load_data())
    test_reward_list = []
    # test for 100 episodes of breakout
    
    for episodes in range(self.test_episodes):
      episode_reward = 0
      obs = env.reset()
      obs = self.preprocess_state(obs)
      self.net_input = obs

      with torch.no_grad():
        while len(self.net_input) != self.warmup_frames:
          action = env.action_space.sample()
          obs, reward, done, info = env.step(action)
          obs = self.preprocess_state(obs)
          self.net_input = torch.cat((self.net_input, obs), dim = 0)        

        while done == False:
          phi_t = self.net_input
          action = self.take_action(phi_t)
          nxt_obs, reward, done, info = env.step(action)
          episode_reward += reward
          preprocesed_next_obs = self.preprocess_state(nxt_obs)
          self.net_input = self.net_input[1:] # remove earliest frame
          self.net_input = torch.cat((self.net_input, preprocesed_next_obs), dim = 0) # add latest
      
      test_reward_list.append(episode_reward)

    return float(np.mean(test_reward_list))
        
  def save_data(self):
    torch.save(self.net.state_dict(),'/content/drive/MyDrive/atari_model_weights'+'/'+'model.pth')
    return

  def load_data(self):
    return torch.load('/content/drive/MyDrive/atari_model_weights'+'/'+'model.pth')

  def log(self):
    print(self.stepcount)
    self.save_data()
    #self.reward_list=[np.mean(self.reward_list[i-100:i]) for i in range(100,len(self.reward_list))]
    plt.plot(self.reward_list)
    plt.savefig('/content/drive/MyDrive/atari_model_weights'+'/'+'train'+'.jpg')

  

In [ ]:
atari = agent(pre_trained_model=True)

In [ ]:
wandb.config = {
  "learning_rate": 0.0001,
  "epochs": 100,
  "batch_size": 32
}

In [ ]:
atari.train()

In [ ]:
atari.test()